# CS188 Assignment 1-2: Linear Regression & Logistic Regression & Softmax Regression

Before we start, please put your name and UD in following format

: Firstname LASTNAME, #00000000   //   e.g.) Yining Hong, #000000000

**Your Answer:**   
Your NAME, #XXXXXXXX

In this notebook you will implement a Linear Regression & Logistic Regression on the tinyplaces dataset you created.

Recall that the Linear Regression does the following:
- Use linear function to model the relationship between an input and an output
- Use loss function to minimize the weight and bias of the linear function. In this assignment, we use Mean Squared Error as the loss function.

Logistic Regression takes a step further by adding a logistic function to the linear output. Here, we use Cross-Entropy Loss to optimize logistic regression.

Logistic Regression and Linear regression are for binary classification. For multi-class classification, we use Softmax Regression. Softmax Regression is an extension for linear regression where we add a softmax function to the linear output, and use NLL-loss to optimize.


The goals of this exercise are to go through a simple example of the data-driven image classification pipeline, and also to practice writing efficient, vectorized code in [PyTorch](https://pytorch.org/).

you may not use any functions from torch.nn or torch.nn.functional.

# Setup Code
Before getting started we need to run some boilerplate code to set up our environment. You'll need to rerun this setup code each time you start the notebook.

First, run this cell load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload) extension. This allows us to edit `.py` source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [ ]:
%load_ext autoreload
%autoreload 2

### Google Colab Setup
Next we need to run a few commands to set up our environment on Google Colab. If you are running this notebook on a local machine you can skip this section.

Run the following cell to mount your Google Drive. Follow the link, sign in to your Google account (the same account you used to store this notebook!) and copy the authorization code into the text box that appears below.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a 188 folder and put all the files under A1 folder, then '188/Assignment1'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '188/Assignment1'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Assignment1'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Once you have successfully mounted your Google Drive and located the path to this assignment, run th following cell to allow us to import from the `.py` files of this assignment. If it works correctly, it should print the message:

```
Hello from regression.py!
```

as well as the last edit time for the file `pytorch101.py`.

In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

from regression import hello
hello()

path = os.path.join(GOOGLE_DRIVE_PATH, 'regression.py')
edit_time = time.ctime(os.path.getmtime(path))
print('regression.py last edited on %s' % edit_time)

## Setup code
Run some setup code for this notebook: Import some useful packages and increase the default figure size.

In [ ]:
import torch
import torchvision
import cs188
import matplotlib.pyplot as plt
import statistics

plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 16

We will run the codes on GPU. Go to:
Runtime->Change Runtime Type->Hardware Accelerator->GPU

In [ ]:
if torch.cuda.is_available:
  print('Good to go!')
else:
  print('Please set GPU via Edit -> Notebook Settings.')

# Linear Regression

We will implement linear regression step by step. Remember that for linear regression:

$$y = WX+b$$

In our case, $X$ is the 3072-dim image pixels. Thus, the weight $W$ should be compatible with the dim of $X$.

## Initialize Weights and Biases and Implement Linear Function
You will implement the linear function and the predict function for the Linear Classification class in `regression.py`.

If the prediction score > 0.5, we consider the image to be of outdoor category. Otherwise, we consider it to be indoor category.

In [ ]:
from regression import *

torch.manual_seed(0)
x_train, y_train, x_val, y_val = cs188.data.tinyplaces(GOOGLE_DRIVE_PATH, use_gpu=True)

regression_model = LinearRegression()
y_pred = regression_model.predict(x_train).squeeze()
print ("output size %s" %(str(y_pred.size())))
print ("train accuracy")
acc = regression_model.check_accuracy(x_train, y_train)
print ("validation accuracy")
acc = regression_model.check_accuracy(x_val, y_val)

Since we initialize the weights and biases to be zero, the outputs should also be zero. Therefore, you should get 50% accuracy here. 

## Find the best W using one-step calculation

As taught in class, for linear regression, the best weight W (or $\hat{\beta}$) can be calculated directly using the training data. Now, implement the calculate_param function in `regression.py` Linear Regression class to modify the weight W.

In [ ]:
from regression import *

torch.manual_seed(0)
x_train, y_train, x_val, y_val = cs188.data.tinyplaces(GOOGLE_DRIVE_PATH)

regression_model = LinearRegression()

regression_model.calculate_param(x_train, y_train)
regression_model.b = regression_model.b.cpu()

y_pred = regression_model.predict(x_train).squeeze()
print ("output size %s" %(str(y_pred.size())))
print ("train accuracy")
acc = regression_model.check_accuracy(x_train, y_train)
print ("validation accuracy")
acc = regression_model.check_accuracy(x_val, y_val)

I can get ~80% train accuracy and ~70% validation accuracy. You should get the similar results.

## Find the best W by gradient descent

Previously, we could calculate the estimated W given the input and output. This is because the linear function is simple to optimize. However, a more general way to find the best parameters is to using gradient descent, as taught in class.

For gradient descent, we need to:

    Compute the losses between the predicted labels and ground-truth labels.
    Back-propagate the losses to get the gradients of the parameters.
    Get the parameters for the next iteration by subtracting the gradient      mutlipled by learning rate.

You will only need to implement the first step as we have implemented the rest of the steps for you.

### Compute losses

We can compare the predictions with the actual targets, using the following method:

    Calculate the difference between the two matrices (preds and targets).
    Square all elements of the difference matrix to remove negative values.
    Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE).

Please implement this in the ``get_loss() function``

In [ ]:
from regression import *

torch.manual_seed(0)
x_train, y_train, x_val, y_val = cs188.data.tinyplaces(GOOGLE_DRIVE_PATH, use_gpu=True)

regression_model = LinearRegression()
y_pred = regression_model.predict(x_train).squeeze()
loss = regression_model.get_loss(y_pred, y_train)
print ("loss is %f"%loss)


The resulting number is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

Since we initialize the weight as zero, all outputs are zero. Therefore, the loss is 0.5 here. (Because it is the mean of the squared loss of the differences between preds and targets. )

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have requires_grad set to True.

In [ ]:
loss.backward()

The gradients are stored in the .grad property of the respective tensors.

In [ ]:
# Gradients for weights
print(regression_model.W.grad)
# Gradients for bias
print(regression_model.b.grad)


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent algorithm, by adjust the weights by subtracting a small quantity proportional to the gradient

In [ ]:
# Adjust weights & reset gradients
with torch.no_grad():
    regression_model.W -= regression_model.W.grad * 1e-4
    regression_model.b -= regression_model.b.grad * 1e-4
    regression_model.W.grad.zero_()
    regression_model.b.grad.zero_()

We now calculate the new loss in the next iteration

In [ ]:
# Calculate loss
y_pred = regression_model.predict(x_train).squeeze()
loss = regression_model.get_loss(y_pred, y_train)
acc = regression_model.check_accuracy(x_val, y_val)
print ("new loss is %f"%loss)


## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.


In [ ]:
from regression import *

x_train, y_train, x_val, y_val = cs188.data.tinyplaces(GOOGLE_DRIVE_PATH, use_gpu=True)

regression_model = LinearRegression()

losses = []

linear_val_accs = []
best_W = None
best_acc = 0

for i in range(500000):
    y_pred = regression_model.predict(x_train).squeeze()
    acc = regression_model.check_accuracy(x_val, y_val, quiet=True)
    linear_val_accs.append(acc)

    loss = regression_model.get_loss(y_pred, y_train)
    losses.append(loss)
    loss.backward()

    if acc > best_acc:
      best_W = regression_model.W
      best_acc = acc

    if i % 10000 == 0:
      print ("%d/500000 iteration"%i)
      print ("validation accuracy is %f"%acc)

    with torch.no_grad():
      regression_model.W -= regression_model.W.grad * 1e-4
      regression_model.b -= regression_model.b.grad * 1e-4
      regression_model.W.grad.zero_()
      regression_model.b.grad.zero_()

I can get ~80% accuracy. Can you beat me?

# Logistic Regression

In `regression.py`, please implement the following functions:

Logistic Regression extends Linear Regression by adding a sigmoid function:
$$\frac{1}{1+e^{-x}}$$
Implement the sigmoid function and the predict function of the Logistic Regression class. 
Then, implement the cross entropy loss :
$$-{(y\log(p) + (1 - y)\log(1 - p))}$$

We will run the results of logistic regression below.

In [ ]:
from regression import *

x_train, y_train, x_val, y_val = cs188.data.tinyplaces(GOOGLE_DRIVE_PATH, use_gpu=True)

regression_model = LogisticRegression()

losses = []
logistic_val_accs = []
best_W = None
best_acc = 0

for i in range(500000):
    y_pred = regression_model.predict(x_train).squeeze()
    acc = regression_model.check_accuracy(x_val, y_val, quiet=True)
    logistic_val_accs.append(acc)

    loss = regression_model.get_loss(y_pred, y_train)
    losses.append(loss)
    loss.backward()

    if acc > best_acc:
      best_W = regression_model.W
      best_acc = acc

    if i % 10000 == 0:
      print ("%d/500000 iteration"%i)

      print ("test accuracy is %f"%acc)

    with torch.no_grad():
      regression_model.W -= regression_model.W.grad * 1e-4
      regression_model.b -= regression_model.b.grad * 1e-4
      regression_model.W.grad.zero_()
      regression_model.b.grad.zero_()

I can get ~83% accuracy. Can you beat me?

### Visualize the accuracies

In [ ]:
plt.plot(linear_val_accs, label='linear regression')
plt.plot(logistic_val_accs, label='logistic regression')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Accuracy (%)')
plt.show()

### Visualize the weights

Visualize the learned weights for each class. If your implementation is right, the visualization should be close to the average of the images.

In [ ]:
w = best_W.reshape(3, 32, 32)
w = w.transpose(0, 2).transpose(1, 0)

w_min, w_max = torch.min(w), torch.max(w)

wimg = 255.0 * (w - w_min) / (w_max - w_min)

plt.imshow(wimg.type(torch.uint8).cpu())
plt.axis('off')

## Softmax Regression

You need to implement the following functions in `regression.py`:

Softmax is a multi-class classification extension from Logistic Regression . Implement the softmax function:
$$\frac{e^{z_{i}}}{\sum_{j=1}^K e^{z_{j}}} \ \ \ for\ i=1,2,\dots,K$$

For softmax regression, the loss function is the NLL (negative log likelihood) loss. Implement the loss function :
$$-{\log(p(y))}$$
Here $p(y)$ is a scaler instead of vector. It is the value of the single dimension where the ground truth $y$ lies. It is thus equivalent to cross entropy

Implement the predict function in Softmax Regression class.

In [ ]:
from regression import *

x_train, y_train, x_test, y_test = cs188.data.tinyplaces(GOOGLE_DRIVE_PATH, binary=False, use_gpu=True)

regression_model = SoftmaxRegression()

losses = []

best_W = None
best_acc = 0

for i in range(1000000):
    y_pred = regression_model.predict(x_train).squeeze()
    acc = regression_model.check_accuracy(x_test, y_test, quiet=True)

    loss = regression_model.get_loss(y_pred, y_train)
    losses.append(loss)
    loss.backward()

    if acc > best_acc:
      best_W = regression_model.W
      best_acc = acc

    if i % 10000 == 0:
      print ("%d/500000 iteration"%i)

      print ("test accuracy is %f"%acc)

    with torch.no_grad():
      regression_model.W -= regression_model.W.grad * 1e-4
      regression_model.b -= regression_model.b.grad * 1e-4
      regression_model.W.grad.zero_()
      regression_model.b.grad.zero_()

### Visualize Weight

Now, visualize the weight for each class. If your implementation is right, the weight of each class should be equivalent to the average of the images of that class.

In [ ]:
w = best_W.reshape(3, 32, 32, 20)
w = w.transpose(0, 2).transpose(1, 0)

w_min, w_max = torch.min(w), torch.max(w)

classes = ['bathroom', 'bedroom', 'bookstore', 'classroom', 'dining_room', 'food_court', 'kitchen', 'lobby', 'living_room', 'office', 'baseball_field', 'bridge', 'campsite', 'canyon', 'coast', 'fountain', 'highway', 'playground', 'mountain', 'rainforest']
for i in range(20):
  plt.subplot(4,5, i + 1)

  # Rescale the weights to be between 0 and 255
  wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
  plt.imshow(wimg.type(torch.uint8).cpu())
  plt.axis('off')
  plt.title(classes[i])

# What's the difference between Linear Regression, Logistic Regression and Softmax Regression? (Write down your answer below)

**Your Answer**: [fill in the answer here]